## Parsing the Lending Club data

In trying to reproduce the analysis in [Venka's blog post](https://juliacomputing.com/blog/2017/08/22/lendingclub-demo-blog.html) I was unable to get some of it working.  In this notebook I will show the steps that I was able to use to parse the data sets.

I suspect that some of my problem has to do with changes in the data sets.  In particular, there are now 137 fields in each of the CSV files and some further anonymizing appears to have been done.  The `id` and `member_id` fields are always empty strings and the `zip_code` field in now of the form `123xx` instead of `12345`.  This means it can't be parsed as an integer.

The files have been augmented with a few lines of information about totals at the bottom.

### Downloading the data

The files can be obtained in a `.csv.zip` format at the [Lending Club site](https://www.lendingclub.com/info/download-data.action).
After unzipping I ran a `sed` command like
```sh
~/LendingTree/files⟫ unzip ~/Downloads/LoanStats3a.csv.zip
Archive:  /home/bates/Downloads/LoanStats3a.csv.zip
  inflating: LoanStats3a.csv
~/LendingTree/files⟫ wc LoanStats3a.csv
   42544  2667455 41466518 LoanStats3a.csv
~/LendingTree/files⟫ sed -i -n -e '/^\"/p' LoanStats3a.csv
~/LendingTree/files⟫ wc LoanStats3a.csv
   42536  2667426 41466304 LoanStats3a.csv
```
to strip any lines that did not begin with the `"` character.

This leaves a total of 1,524,087 records in the CSV files.
```sh
~/LendingTree/files⟫ wc -l LoanStats*.csv
    133888 LoanStats_2016Q1.csv
     97855 LoanStats_2016Q2.csv
     99121 LoanStats_2016Q3.csv
    103547 LoanStats_2016Q4.csv
     96780 LoanStats_2017Q1.csv
    105452 LoanStats_2017Q2.csv
     42536 LoanStats3a.csv
    188182 LoanStats3b.csv
    235630 LoanStats3c.csv
    421096 LoanStats3d.csv
   1524087 total
```

### Using TextParse.csvread on a single file

The most difficult part is creating the appropriate column parsers.  All of the fields are enclosed in quotes. I ended up using

In [1]:

using TextParse: csvread, Quoted, StrRange

const parser = Dict(
    "id"                              => Quoted(Nullable{Int}, required=true),
    "member_id"                       => Quoted(Nullable{Int}, required=true),
    "loan_amnt"                       => Quoted(Nullable{Int}, required=true),
    "funded_amnt"                     => Quoted(Nullable{Int}, required=true),
    "funded_amnt_inv"                 => Quoted(Nullable{Int}, required=true),
    "term"                            => Quoted(StrRange, required=true),
    "int_rate"                        => Quoted(StrRange, required=true),
    "installment"                     => Quoted(Nullable{Float64}, required=true),
    "grade"                           => Quoted(StrRange, required=true),
    "sub_grade"                       => Quoted(StrRange, required=true),
    "emp_title"                       => Quoted(StrRange, required=true),
    "emp_length"                      => Quoted(StrRange, required=true),
    "home_ownership"                  => Quoted(StrRange, required=true),
    "annual_inc"                      => Quoted(Nullable{Float64}, required=true),
    "verification_status"             => Quoted(StrRange, required=true),
    "issue_d"                         => Quoted(StrRange, required=true),
    "loan_status"                     => Quoted(StrRange, required=true),
    "pymnt_plan"                      => Quoted(StrRange, required=true),
    "url"                             => Quoted(StrRange, required=true),
    "desc"                            => Quoted(StrRange, required=true),
    "purpose"                         => Quoted(StrRange, required=true),
    "title"                           => Quoted(StrRange, required=true),
    "zip_code"                        => Quoted(StrRange, required=true),
    "addr_state"                      => Quoted(StrRange, required=true),
    "dti"                             => Quoted(Nullable{Float64}, required=true),
    "delinq_2yrs"                     => Quoted(Nullable{Int}, required=true),
    "earliest_cr_line"                => Quoted(StrRange, required=true),
    "inq_last_6mths"                  => Quoted(Nullable{Int}, required=true),
    "mths_since_last_delinq"          => Quoted(Nullable{Int}, required=true),
    "mths_since_last_record"          => Quoted(Nullable{Int}, required=true),
    "open_acc"                        => Quoted(Nullable{Int}, required=true),
    "pub_rec"                         => Quoted(Nullable{Int}, required=true),
    "revol_bal"                       => Quoted(Nullable{Int}, required=true),
    "revol_util"                      => Quoted(StrRange, required=true),
    "total_acc"                       => Quoted(Nullable{Int}, required=true),
    "initial_list_status"             => Quoted(StrRange, required=true),
    "out_prncp"                       => Quoted(Nullable{Float64}, required=true),
    "out_prncp_inv"                   => Quoted(Nullable{Float64}, required=true),
    "total_pymnt"                     => Quoted(Nullable{Float64}, required=true),
    "total_pymnt_inv"                 => Quoted(Nullable{Float64}, required=true),
    "total_rec_prncp"                 => Quoted(Nullable{Float64}, required=true),
    "total_rec_int"                   => Quoted(Nullable{Float64}, required=true),
    "total_rec_late_fee"              => Quoted(Nullable{Float64}, required=true),
    "recoveries"                      => Quoted(Nullable{Float64}, required=true),
    "collection_recovery_fee"         => Quoted(Nullable{Float64}, required=true),
    "last_pymnt_d"                    => Quoted(StrRange, required=true),
    "last_pymnt_amnt"                 => Quoted(Nullable{Float64}, required=true),
    "next_pymnt_d"                    => Quoted(StrRange, required=true),
    "last_credit_pull_d"              => Quoted(StrRange, required=true),
    "collections_12_mths_ex_med"      => Quoted(Nullable{Int}, required=true),
    "mths_since_last_major_derog"     => Quoted(Nullable{Int}, required=true),
    "policy_code"                     => Quoted(Nullable{Int}, required=true),
    "application_type"                => Quoted(StrRange, required=true),
    "annual_inc_joint"                => Quoted(Nullable{Float64}, required=true),
    "dti_joint"                       => Quoted(Nullable{Float64}, required=true),
    "verification_status_joint"       => Quoted(StrRange, required=true),
    "acc_now_delinq"                  => Quoted(Nullable{Int}, required=true),
    "tot_coll_amt"                    => Quoted(Nullable{Int}, required=true),
    "tot_cur_bal"                     => Quoted(Nullable{Int}, required=true),
    "open_acc_6m"                     => Quoted(Nullable{Int}, required=true),
    "open_il_6m"                      => Quoted(Nullable{Int}, required=true),
    "open_il_12m"                     => Quoted(Nullable{Int}, required=true),
    "open_il_24m"                     => Quoted(Nullable{Int}, required=true),
    "mths_since_rcnt_il"              => Quoted(Nullable{Int}, required=true),
    "total_bal_il"                    => Quoted(Nullable{Int}, required=true),
    "il_util"                         => Quoted(Nullable{Float64}, required=true),
    "open_rv_12m"                     => Quoted(Nullable{Int}, required=true),
    "open_rv_24m"                     => Quoted(Nullable{Int}, required=true),
    "max_bal_bc"                      => Quoted(Nullable{Int}, required=true),
    "all_util"                        => Quoted(Nullable{Float64}, required=true),
    "total_rev_hi_lim"                => Quoted(Nullable{Int}, required=true),
    "inq_fi"                          => Quoted(Nullable{Int}, required=true),
    "total_cu_tl"                     => Quoted(Nullable{Int}, required=true),
    "inq_last_12m"                    => Quoted(Nullable{Int}, required=true),
    "acc_open_past_24mths"            => Quoted(Nullable{Int}, required=true),
    "avg_cur_bal"                     => Quoted(Nullable{Int}, required=true),
    "bc_open_to_buy"                  => Quoted(Nullable{Int}, required=true),
    "bc_util"                         => Quoted(Nullable{Float64}, required=true),
    "chargeoff_within_12_mths"        => Quoted(Nullable{Int}, required=true),
    "delinq_amnt"                     => Quoted(Nullable{Int}, required=true),
    "mo_sin_old_il_acct"              => Quoted(Nullable{Int}, required=true),
    "mo_sin_old_rev_tl_op"            => Quoted(Nullable{Int}, required=true),
    "mo_sin_rcnt_rev_tl_op"           => Quoted(Nullable{Int}, required=true),
    "mo_sin_rcnt_tl"                  => Quoted(Nullable{Int}, required=true),
    "mort_acc"                        => Quoted(Nullable{Int}, required=true),
    "mths_since_recent_bc"            => Quoted(Nullable{Int}, required=true),
    "mths_since_recent_bc_dlq"        => Quoted(Nullable{Int}, required=true),
    "mths_since_recent_inq"           => Quoted(Nullable{Int}, required=true),
    "mths_since_recent_revol_delinq"  => Quoted(Nullable{Int}, required=true),
    "num_accts_ever_120_pd"           => Quoted(Nullable{Int}, required=true),
    "num_actv_bc_tl"                  => Quoted(Nullable{Int}, required=true),
    "num_actv_rev_tl"                 => Quoted(Nullable{Int}, required=true),
    "num_bc_sats"                     => Quoted(Nullable{Int}, required=true),
    "num_bc_tl"                       => Quoted(Nullable{Int}, required=true),
    "num_il_tl"                       => Quoted(Nullable{Int}, required=true),
    "num_op_rev_tl"                   => Quoted(Nullable{Int}, required=true),
    "num_rev_accts"                   => Quoted(Nullable{Int}, required=true),
    "num_rev_tl_bal_gt_0"             => Quoted(Nullable{Int}, required=true),
    "num_sats"                        => Quoted(Nullable{Int}, required=true),
    "num_tl_120dpd_2m"                => Quoted(Nullable{Int}, required=true),
    "num_tl_30dpd"                    => Quoted(Nullable{Int}, required=true),
    "num_tl_90g_dpd_24m"              => Quoted(Nullable{Int}, required=true),
    "num_tl_op_past_12m"              => Quoted(Nullable{Int}, required=true),
    "pct_tl_nvr_dlq"                  => Quoted(Nullable{Float64}, required=true),
    "percent_bc_gt_75"                => Quoted(Nullable{Float64}, required=true),
    "pub_rec_bankruptcies"            => Quoted(Nullable{Int}, required=true),
    "tax_liens"                       => Quoted(Nullable{Int}, required=true),
    "tot_hi_cred_lim"                 => Quoted(Nullable{Int}, required=true),
    "total_bal_ex_mort"               => Quoted(Nullable{Int}, required=true),
    "total_bc_limit"                  => Quoted(Nullable{Int}, required=true),
    "total_il_high_credit_limit"      => Quoted(Nullable{Int}, required=true),
    "revol_bal_joint"                 => Quoted(Nullable{Int}, required=true),
    "sec_app_earliest_cr_line"        => Quoted(StrRange, required=true),
    "sec_app_inq_last_6mths"          => Quoted(StrRange, required=true),
    "sec_app_mort_acc"                => Quoted(StrRange, required=true),
    "sec_app_open_acc"                => Quoted(StrRange, required=true),
    "sec_app_revol_util"              => Quoted(StrRange, required=true),
    "sec_app_open_il_6m"              => Quoted(StrRange, required=true),
    "sec_app_num_rev_accts"           => Quoted(StrRange, required=true),
    "sec_app_chargeoff_within_12_mths"=> Quoted(StrRange, required=true),
    "sec_app_collections_12_mths_ex_med"=> Quoted(StrRange, required=true),
    "sec_app_mths_since_last_major_derog"=> Quoted(StrRange, required=true),
    "hardship_flag"                   => Quoted(StrRange, required=true),
    "hardship_type"                   => Quoted(StrRange, required=true),
    "hardship_reason"                 => Quoted(StrRange, required=true),
    "hardship_status"                 => Quoted(StrRange, required=true),
    "deferral_term"                   => Quoted(StrRange, required=true),
    "hardship_amount"                 => Quoted(Nullable{Int}, required=true),
    "hardship_start_date"             => Quoted(StrRange, required=true),
    "hardship_end_date"               => Quoted(StrRange, required=true),
    "payment_plan_start_date"         => Quoted(StrRange, required=true),
    "hardship_length"                 => Quoted(StrRange, required=true),
    "hardship_dpd"                    => Quoted(StrRange, required=true),
    "hardship_loan_status"            => Quoted(StrRange, required=true),
    "orig_projected_additional_accrued_interest" => Quoted(StrRange, required=true),
    "hardship_payoff_balance_amount"  => Quoted(StrRange, required=true),
    "hardship_last_payment_amount"    => Quoted(StrRange, required=true));

In [2]:
lending3b = csvread("files/LoanStats3b.csv", escapechar='"',
    colparsers = parser, header_exists=true,
    pooledstrings = true);

Some of the columns could be parsed without allowing `Nullable`.

In [3]:
any(isnull, lending3b[1][3])

false

Others are all nulls, because of the anonymization.

In [4]:
all(isnull, lending3b[1][1])

true